# Automated Hyper-Parameter Tuning

In [ ]:
## tensorflow
import tensorflow as tf

## sklearn
from sklearn.svm import SVC
from sklearn import datasets
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

## hyperopt
from hyperopt import tpe, hp, fmin
from hyperopt.mongoexp import MongoTrials

## hyperas
from hyperas import optim
from hyperas.distributions import choice, uniform

## 1. sklearn models

### 1.1 single-machine tuning

In [ ]:
iris = datasets.load_iris()
x = iris.data
y = iris.target
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [ ]:
def objective_func(args):
    if args['model']==KNeighborsClassifier:
        n_neighbors = args['param']['n_neighbors']
        algorithm = args['param']['algorithm']
        leaf_size = args['param']['leaf_size']
        metric = args['param']['metric']
        clf = KNeighborsClassifier(n_neighbors=n_neighbors,
                               algorithm=algorithm,
                               leaf_size=leaf_size,
                               metric=metric,
                               )
    elif args['model']==SVC:
        C = args['param']['C']
        kernel = args['param']['kernel']
        degree = args['param']['degree']
        gamma = args['param']['gamma']
        clf = SVC(C=C, kernel=kernel, degree=degree,gamma=gamma)
    
    clf.fit(x_train,y_train)
    y_pred_test = clf.predict(x_test)
    loss = mean_squared_error(y_test,y_pred_test)
    print("Test Score:",clf.score(x_test,y_test))
    print("Train Score:",clf.score(x_train,y_train))
    print("\n=================")
    return loss

In [ ]:
space = hp.choice('classifier',[
        {'model': KNeighborsClassifier,
        'param': 
             {'n_neighbors':hp.choice('n_neighbors',range(3,11)),
              'algorithm':hp.choice('algorithm',['ball_tree','kd_tree']),
              'leaf_size':hp.choice('leaf_size',range(1,50)),
              'metric':hp.choice('metric', ["euclidean","manhattan", "chebyshev","minkowski"])
             }
        },
        {'model': SVC,
        'param':
         {'C':hp.lognormal('C',0,1),
          'kernel':hp.choice('kernel',['rbf','poly','rbf','sigmoid']),
          'degree':hp.choice('degree',range(1,15)),
          'gamma':hp.uniform('gamma',0.001,10000)}
        }
        ])

In [ ]:
best_classifier = fmin(objective_func,space,
                        algo=tpe.suggest,max_evals=100)
print(best_classifier)

### 1.2 distributed tuning

Distributed tuning is achived via MongoDB which acts as a job broker. The main program (this notebook) spawns training jobs with proposed set of hyper-parameters to MongoDB. On the other side, workers take jobs one by one and update training performance.

In [ ]:
trials = MongoTrials('mongo://localhost:27017/iris/jobs',
                     exp_key='exp6')

In [ ]:
best_classifier = fmin(objective_func,space,trials=trials,
                        algo=tpe.suggest,max_evals=100)

In [ ]:
print(best_classifier)

## 2. tensorflow models

### 2.1 single-machine tuning

In [ ]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
def objective_func_tf(args):
    
    activ = args['activation']
    dropout = args['dropout']
    model = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=(28, 28)),
      tf.keras.layers.Dense(128, activation=activ),
      tf.keras.layers.Dropout(dropout),
      tf.keras.layers.Dense(10, activation='softmax')
    ])
    
    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    
    model.fit(x_train, y_train, epochs=5)
    loss, accuracy = model.evaluate(x_test,  y_test, verbose=2)
    print("Test Loss:", loss)
    print("Test accuracy:", accuracy)
    print("\n=================")
    return loss

In [ ]:
space_tf = {'activation': hp.choice('activation',['relu','sigmoid']),
         'dropout': hp.uniform('dropout',0,1)
        }

In [ ]:
best_classifier = fmin(objective_func_tf, space_tf,
                        algo=tpe.suggest,max_evals=10)

In [ ]:
best_classifier

### 2.2 distributed tuning

MongoTrials doesn't work well with tensorflow neural network models. It often throws pickle errors. In order to run in distributed fashion, we need to use `hyperas` library. Everything works same on the main program side. On the worker side, we need to copy `temp_model.py` from main program folder over to worker folder.

In [ ]:
def data():
    mnist = tf.keras.datasets.mnist
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    return x_train, y_train, x_test, y_test

In [ ]:
def create_model(x_train, y_train, x_test, y_test):
    
    model = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=(28, 28)),
      tf.keras.layers.Dense(128, activation={{choice(['relu', 'sigmoid'])}}),
      tf.keras.layers.Dropout({{uniform(0, 1)}}),
      tf.keras.layers.Dense(10, activation='softmax')
    ])
    
    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    
    model.fit(x_train, y_train, epochs=5)
    loss, accuracy = model.evaluate(x_test,  y_test, verbose=2)
    print("Test Loss:", loss)
    print("Test accuracy:", accuracy)
    print("\n=================")
    return loss

In [ ]:
trials = MongoTrials('mongo://localhost:27017/tf_mnist/jobs',
                     exp_key='exp2')
best_run, best_model = optim.minimize(model=create_model,
                                      data=data,
                                      algo=tpe.suggest,
                                      max_evals=10,
                                      trials=trials,
                                      notebook_name="tool1. try out hyperopt",
                                      keep_temp=True)

In [ ]:
best_run